<a href="https://colab.research.google.com/github/ogdtg/starter-code-ogdtg/blob/main/_work/01_R_Notebooks/rnb_ogdtg@dek-kb-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Open Government Data, provided by **Canton Thurgau**
*Autogenerated R starter code for dataset with the identifier* **dek-kb-3**

[Link to dataset on data.tg.ch](https://data.tg.ch/explore/dataset/dek-kb-3/information/)


## Dataset
# **Bibliothekenstatistik der Öffentlichen Bibliotheken im Kanton Thurgau (ab 2020)**

## Description

<p>Der Datensatz enthält Kennzahlen zu den öffentlichen Bibliotheken im Kanton Thurgau ab 2020. Dazu zählen Kennzahlen zu den Besuchenden, Beständen, Nutzung oder Mitarbeitenden.</p><p>Die Daten stammen aus der 2021 revidierten Schweizer Bibliotheksstatistik. Bis und mit 2024 (Referenzjahr 2023) wurde die Statistik im Auftrag des Bundesamtes für Statistik (BFS) durchgeführt. Seit 2025 (Referenzjahr 2024) vom Verband Bibliosuisse in Auftrag gegeben (mehr dazu unter https://www.bibliosuisse.ch/aktivitaten/bibliotheksstatistik). </p><p></p><p><b>Hinweise</b>: </p><ul><li>Nicht enthalten im Datensatz sind Schulbibliotheken, welche nicht öffentlich zugänglich sind.</li><li>Fehlende Angaben sind mit NA (no answer) ausgewiesen.</li><li>Die Rücklaufquote ist in den Jahren 2023 und 2024 jeweils nicht 100 %. 2023 fehlen die Angaben der Bibliothek Egnach und 2024 jene der Bibliotheken LuBi Dussnang und Egnach.</li></ul><p></p><p>Hinweis zur Revision der Schweizer Bibliotheksstatistik 2021:</p><p>Die Schweizer Bibliotheksstatistik wurde einer grundlegenden Revision unterzogen und wird erstmals ab 2021 bei allen öffentlich zugänglichen Bibliotheken der Schweiz durchgeführt. Die Anzahl der Variablen wurde deutlich reduziert und ihre Definitionen wurden mit Blick auf moderne Standards (z.B. ISO2789 oder COUNTER 5) hin überarbeitet und festgelegt. Die aktuellen Daten lassen sich daher nur bedingt mit den Daten der Vorjahre vergleichen. </p><p></p><p>Die detaillierten Beschreibungen der einzelnen Variablen finden sich im Dokument 'Referenzen und Definitionen der Variablen' unter https://dam-api.bfs.admin.ch/hub/api/dam/assets/30645434/master.</p><p></p><p>Grafische und tabellarische Auswertungen zu den öffentlichen Bibliotheken im Thurgau ab 2016 finden Sie auch auf der Webseite des Amts für Daten und Statistik unter https://statistik.tg.ch/themen-und-daten/bildung-und-kultur/kultur/bibliotheken.html/7622.</p><p></p><p>Datenquelle: Kantonsbibliothek Kanton Thurgau</p>

## Metadata

- Issued                   : 
- Modified                 : 2025-10-08 12:58:37
- Rights                   : CC0 1.0
- Keywords                 : bibliothek, bibliothekenstatistik, ausleihen, nutzung, medien, bestand, druckschriften, handschriften, karten, zeitungen, zeitschriften, ebooks, audio-visuell, datenbanken

## Load Packages

Every dataset (csv and geodata) on [data.tg.ch](https://data.tg.ch/pages/start/) can be retrieved with the **tgAPI** package. You can download it via [GitHub](https://github.com/ogdtg/tgAPI). Please note that the package has some dependencies, which can result in a lengthy installation process. Therefore, we do not recommend installing the package directly in Google Colab. However, if you intend to run this code on your local machine, we strongly recommend installing the tgAPI package.

In [ ]:
library(httr)
library(jsonlite)
library(dplyr)
library(ggplot2)
library(tidyr)
library(repr)
options(repr.plot.width=25, repr.plot.height=10)
# library(remotes)
# remotes::install.github("ogdtg/tgAPI")

## Download Data



In [ ]:
# Download the Dataset via API
res <- httr::GET("https://data.tg.ch/api/explore/v2.0/catalog/datasets/dek-kb-3/exports/json")
data <- res$content %>%
  rawToChar() %>%
  jsonlite::fromJSON()

# Use tgAPI
# data <- tgAPI::get_dataset("dek-kb-3")

head(data)



## Explore Metadata

In [ ]:
res <- httr::GET("https://data.tg.ch/api/explore/v2.0/catalog/datasets/dek-kb-3")
metadata <- res$content %>%
  rawToChar() %>%
  jsonlite::fromJSON()

# Fields name
cat("\nFields name")
metadata$dataset$fields$name

# Fields description
cat("\nFields description")
metadata$dataset$fields$description

# Fields data type
cat("\nFields data type")
metadata$dataset$fields$type

## Analyze data





In [ ]:
# Look at the dataset and its structure
summary(data)

### Draw Histograms on Numeric Variables

In [ ]:
data %>%
  select_if(is.numeric) %>%
  gather() %>%
  ggplot( aes(value)) +
  geom_histogram(bins = 10, color = "white", fill = "red") +
    facet_wrap(~key, scales = 'free_x') +
    labs(title = "Histogram of dek-kb-3")

### Detect Missing Values

In [ ]:
# detect missing values

missing.values <- data %>%
  gather(key = "key", value = "val") %>%
  mutate(isna = is.na(val)) %>%
  group_by(key) %>%
  mutate(total = n()) %>%
  group_by(key, total, isna) %>%
  summarise(num.isna = n()) %>%
  mutate(pct = num.isna / total * 100)

levels <-
    (missing.values  %>% filter(isna == T) %>% arrange(desc(pct)))$key

percentage.plot <- missing.values %>%
      ggplot() +
        geom_bar(aes(x = reorder(key, desc(pct)),
                     y = pct, fill=isna),
                 stat = 'identity', alpha=0.8) +
      scale_x_discrete(limits = levels) +
      scale_fill_manual(name = "",
                        values = c('steelblue', 'tomato3'), labels = c("Present", "Missing")) +
      coord_flip() +
      labs(title = "Percentage of missing values in dek-kb-3", x =
             'Variable', y = "% of missing values")

percentage.plot

In [ ]:
row.plot <- data %>%
  mutate(id = row_number()) %>%
  gather(-id, key = "key", value = "val") %>%
  mutate(isna = is.na(val)) %>%
  ggplot(aes(key, id, fill = isna)) +
    geom_raster(alpha=0.8) +
    scale_fill_manual(name = "",
        values = c('steelblue', 'tomato3'),
        labels = c("Present", "Missing")) +
    scale_x_discrete(limits = levels) +
    labs(x = "Variable",
           y = "Row Number", title = "Missing values in rows of dek-kb-3") +
    coord_flip()

row.plot

## Continue your code here...


In [ ]:
# Do some data magic with R

------------------------------------------------------------------------

## Questions about the data?

Contact us via [Mail](mailto:ogd@tg.ch) or visit our [GitHub presence](https://github.com/ogdtg)


------------------------------------------------------------------------

OGD-Koordinationsstelle Kanton Thurgau | Grabenstrasse 8 | 8510 Frauenfeld

